In [ ]:
!pip install langchain
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [ ]:
import torch
from langchain_community.llms import HuggingFacePipeline
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

model_id = "google/gemma-2-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=70,  # Limite a resposta para 100 tokens para rapidez
    do_sample=False,      # Permite respostas mais criativas
    temperature=0.0,     # Ajuste a temperatura para controle da aleatoriedade
    repetition_penalty= 1.2    # Penaliza a repetição de sequências (ideal para quebrar o loop
    )

llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"skip_prompt": True } )



tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cpu
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/tmp/ipython-input-2913659430.py:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe, model_kwargs={"skip_prompt": True } )


#### Função de Extração

Responsável pela coleta dos e-mails e parseamento dos dados de compra e venda.

In [ ]:
def criar_prompt_gemma_extrator(email_data, feedback: str = None):

  if feedback and feedback.strip():
    instrucao_feedback = f"""

              "ATENÇÃO: Este é um ciclo de **Autocorreção**. Você já tentou extrair este e-mail antes e o resultado foi rejeitado. "
              "Você DEVE usar o FEEDBACK CRÍTICO abaixo para corrigir o JSON anterior. Não repita o erro."

    -- FEEDBACK --
    {feedback.strip()}

    """
  else:
    instrucao_feedback = ""


  instrucao_sistema = """

        Você é um assistente de extração de dados. Sua tarefa é analisar o corpo de uma mensagem de e-mail e extrair quatro informações-chave: o nome do ativo financeiro, o preço unitário, a quantidade e a data de vencimento.

        Os dados que você está coletando não podem ser diferentes do e-mail enviado

        A resposta deve ser ESTRITAMENTE um dicionário JSON, seguindo este formato:
        {{"Nome": string, "Preco": float, "Qtd": integer, "Vencimento": string}}

        REGRAS FORTES SOBRE DATAS (obrigatórias):
        1) "Vencimento" é a data associada a palavras como: "vencimento", "vencer", "vcto", "maturity", "venc.".
        2) NUNCA use datas associadas a "liquidação", "settlement", "liq.", "D+X".
        3) Se houver várias datas, escolha a que está:
          a) na MESMA frase/linha ou imediatamente ao lado de "vencimento"/"vencer"/"vcto"/"maturity";
          b) e NÃO contenha as palavras de liquidação ("liquidação", "liq").

        Regras:
        - A data de liquidação é uma data próxima a data de hoje
        - A data de vencimento é diferente da data de liquidação
        - A data de vencimento é diferente da data de hoje
        - Se alguma informação não for encontrada, use null.
        - Se o preço estiver em reais (R$), remova o símbolo.
        - A resposta deve ser APENAS o JSON (sem comentários).

        """

  conteudo_user = f"""

          {instrucao_feedback.strip()}

          {instrucao_sistema.strip()}

          Aqui está o e-mail para análise. Extraia os dados:

          --- EMAIL BRUTO ---
          {email_data.strip()}

         """

  #Formato nativo
  prompt_completo = (
          f"<bos><start_of_turn>user\n"
          f"{conteudo_user.strip()}"
          f"<end_of_turn><start_of_turn>model\n"
          f"{{" # Opcional, mas incentiva o modelo a começar imediatamente com a chave JSON
      )

  resposta_bruta = llm.invoke(prompt_completo)
  index_resposta_modelo = resposta_bruta.find('<start_of_turn>model')
  resposta_limpa = resposta_bruta[index_resposta_modelo + 20:]

  return resposta_limpa



#### Função de Verificação

Responsável pela validação dos dados retornados pelo Agente Extrator (Ex: verificação do tipo dos dados retornados pelo json, validação do montante financeiro da operação)

In [ ]:
def criar_prompt_gemma_verificador(return_data):


    instrucao_sistema = """

          Você é um **Agente de Verificação e Crítica**. Sua única função é inspecionar o JSON extraído e o e-mail original para garantir que o JSON é **ESTRITAMENTE VÁLIDO**.

          --- PADRÃO JSON EXIGIDO ---
          O formato deve ser ESTRITAMENTE: {{"Nome": string, "Preco": float, "Qtd": integer, "Vencimento": string}}

          --- REGRAS DE VALIDAÇÃO ---
          1. **Verificação de Nulos:** TODOS os campos ("Nome", "Preco", "Qtd", "Vencimento") são obrigatórios. Se algum campo estiver com o valor `null`, o JSON é REPROVADO.
          2. **Verificação de Tipo/Padrão:**
              - "Preco" e "Qtd" devem ser números (float ou integer) e não strings.
              - O formato da chave deve ser exatamente o exigido (sem caracteres ou chaves extras).
          3. **Verificação de Conteúdo:** O valor de cada campo deve estar semanticamente correto com o EMAIL BRUTO.

          --- REGRA DE SAÍDA CRÍTICA ---
          Sua resposta deve ser **ESTRITAMENTE** uma das duas opções:

          A. **APROVAÇÃO (JSON VÁLIDO):** Se o JSON for 100% válido, responda APENAS "".
          B. **REPROVAÇÃO E FEEDBACK (JSON INVÁLIDO):** Se o JSON falhar em qualquer regra, responda com uma mensagem de **feedback DETALHADA** que será usada por outro Agente para corrigir o JSON. Seu feedback deve começar com a frase: "JSON REPROVADO. Por favor, corrija o seguinte:".

          """

    conteudo_user = f"""

            {instrucao_sistema.strip()}

            Aqui está o json para análise. Faça a análise:

            --- JSON ---
            {return_data.strip()}

         """

  #Formato nativo
    prompt_completo = (
          f"<bos><start_of_turn>user\n"
          f"{conteudo_user.strip()}"
          f"<end_of_turn><start_of_turn>model\n"
          f"{{" # Opcional, mas incentiva o modelo a começar imediatamente com a chave JSON
      )

    resposta_bruta = llm.invoke(prompt_completo)
    index_resposta_modelo = resposta_bruta.find('<start_of_turn>model')
    resposta_limpa = resposta_bruta[index_resposta_modelo + 20:]

    return resposta_limpa



#### Função de Orquestração

Responsável pela execução e comunicação das funções durante o processamento do e-mail. A função está configurada para fazer 3 tentativas em caso de um retorno fora do padrão esperado pelo verificador.

In [ ]:
def orquestrador(e-mail):

  num_tentativas = 3

  return_extrator = criar_prompt_gemma_extrator(e-mail)
  feedback = criar_prompt_gemma_verificador(return_extrator)

  if feedback is not "":
      for i in range(num_tentativas + 1):
        return_extrator = criar_prompt_gemma_extrator(e-mail)
        return_verificador =
        if return_verificador == "":
          break


## Teste de Execução

In [ ]:
email_teste = (
    "Prezado cliente, confirmamos a operação de Títulos: "
    "COMPRA de 8000 unidades de LTN que vai vencer em 18/06/2035. PU de R$ 19,50. "
    "Liquidação marcada para 12/03/2026. Obrigado."
)

email_teste2 = (
    """Senhores,

    Segue trade:

      VENDE:
      NTN-B 15/05/2027
      QTDE: 30.000
      TAXA: 8,9210%
      PU: 4.480,152199
      FIN: R$ 1000,00
      LIQ: 08/10/2025 """
)

print(criar_prompt_gemma_extrator(email_teste))



{
  "Nome": "LTN",
  "Preco": 19.5,
  "Qtd": 8000,
  "Vencimento": "18/06/2035"
}
